# Set-up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os


In [2]:
DIR = '/content/drive/My Drive/ml-mini-projects/black-friday'

# Framing of the Problem

## Challenge:

The following dataset contains information from stores on a Black Friday, which sometimes refer to the Thanksgiving Friday in the US. You are tasked to build a model to predict the purchase amount of customers against various products.

**Evaluation Metric:** RMSE <br>
**Top scorer score:** 2372.0034421821

**Notes:** <br>
This challenge and dataset was sourced from [Analytics Vidhya](https://datahack.analyticsvidhya.com/contest/black-friday/?utm_source=facebook.com&utm_medium=social)


**Approach:** Regression Problem

# Taking Quick Glances

In [3]:
df = pd.read_csv(os.path.join(DIR, 'train.csv'))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


**Observations:**
* It has a total of 550, 068 entries with 12 columns
   * Not so many features. Let's not do feature selection first
* Data types
   * User_ID, Product_ID 
      * not sure if this will help; try dropping
   * Gender, City Category as strings
      * hot encode this
   * Age, Stay In Current City Years are unexpectedly written as strings
      * Label Encoding - try converting to int or float (check unique values to decide)
   * Occupation is int (so already label encoded)
      * This may be placed for a reason. Let's try using this before hot encoding
   * Marital Status is already in its correct form
   * Product Category 1 and 2 are ints and 3 are floats.
      * can we convert Product Category 3 as int too?
      * also let's try to use this(label encoded) before doing one-hot
   * Purchase
      * does this represent the actual or in thousands or what? (no info to verify this)
* Nulls
   * Product Category 2 and 3 have many nulls


1. Let's first deal with the nulls

In [5]:
pd.concat([pd.DataFrame(df.isna().sum(), columns=['count']),(pd.DataFrame((df.isna().sum()/df.shape[0])*100, columns=['percentage']))], axis=1)

,count,percentage
User_ID,0,0.000000
Product_ID,0,0.000000
Gender,0,0.000000
Age,0,0.000000
Occupation,0,0.000000
City_Category,0,0.000000
Stay_In_Current_City_Years,0,0.000000
Marital_Status,0,0.000000
Product_Category_1,0,0.000000
Product_Category_2,173638,31.566643


In [6]:
# what if we drop Product_Category_2 only? How will that affect Product Category 3?
df_dum = df.dropna(subset=['Product_Category_2'])
df_dum.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2                 0
Product_Category_3            209609
Purchase                           0
dtype: int64

**Possible approach for NaNs**
1. If we drop all the nulls,
   * we will only be left with 166,821 which is 30% of the original dataset.
2. If we drop only Product_Category_2, 
   * 173, 638 are deducted so it means, those that are null in Product_Category_2 are also null in Product_Category_3
3. Replace nulls with 0s
   * I'll be trying this approach

Further: We can also see the nature of these rows with null later, so we will know its impact on our work.

In [7]:
# let's put it to a new dataframe since we can try droppin
df1 = df.fillna(0)

In [8]:
df1.isna().sum() # sanity check

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

In [9]:
df1.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,0.0,0.0,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,0.0,0.0,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,0.0,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,0.0,0.0,7969


2. Are there duplicates? none.

In [10]:
# are there duplicates?
df1.duplicated().sum()

0

In [11]:
# numerical data
df1.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,6.735436,3.841941,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,6.215492,6.250712,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,0.000000,0.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,0.000000,0.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,5.000000,0.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,14.000000,8.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


3. Convert Product_Category_3

In [12]:
# look on unique values per column and number of unique values
for col in list(df1.columns):
  print(col, df1[col].unique(), len(df1[col].unique()))

User_ID [1000001 1000002 1000003 ... 1004113 1005391 1001529] 5891
Product_ID ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853'] 3631
Gender ['F' 'M'] 2
Age ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25'] 7
Occupation [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6] 21
City_Category ['A' 'C' 'B'] 3
Stay_In_Current_City_Years ['2' '4+' '3' '1' '0'] 5
Marital_Status [0 1] 2
Product_Category_1 [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19] 20
Product_Category_2 [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.] 18
Product_Category_3 [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.] 16
Purchase [ 8370 15200  1422 ...   135   123   613] 18105


Product_Category_2 and Product_Category_3 are actually integers. Let's convert them to integers

In [13]:
df1 = df1.astype({'Product_Category_2':'int64', 'Product_Category_3':'int64'})

In [14]:
df1.info() # sanity check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   User_ID                     550068 non-null  int64 
 1   Product_ID                  550068 non-null  object
 2   Gender                      550068 non-null  object
 3   Age                         550068 non-null  object
 4   Occupation                  550068 non-null  int64 
 5   City_Category               550068 non-null  object
 6   Stay_In_Current_City_Years  550068 non-null  object
 7   Marital_Status              550068 non-null  int64 
 8   Product_Category_1          550068 non-null  int64 
 9   Product_Category_2          550068 non-null  int64 
 10  Product_Category_3          550068 non-null  int64 
 11  Purchase                    550068 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 50.4+ MB


Other attributes' data type won't be changed for visualization but will be changed for data prep for ML

# Approach 1

## Prepocessing

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   User_ID                     550068 non-null  int64 
 1   Product_ID                  550068 non-null  object
 2   Gender                      550068 non-null  object
 3   Age                         550068 non-null  object
 4   Occupation                  550068 non-null  int64 
 5   City_Category               550068 non-null  object
 6   Stay_In_Current_City_Years  550068 non-null  object
 7   Marital_Status              550068 non-null  int64 
 8   Product_Category_1          550068 non-null  int64 
 9   Product_Category_2          550068 non-null  int64 
 10  Product_Category_3          550068 non-null  int64 
 11  Purchase                    550068 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 50.4+ MB


In [16]:
# let's drop User_ID and Product_ID
df_app1 = df1.drop(columns=['User_ID', 'Product_ID'], axis=1)
df_app1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Gender                      550068 non-null  object
 1   Age                         550068 non-null  object
 2   Occupation                  550068 non-null  int64 
 3   City_Category               550068 non-null  object
 4   Stay_In_Current_City_Years  550068 non-null  object
 5   Marital_Status              550068 non-null  int64 
 6   Product_Category_1          550068 non-null  int64 
 7   Product_Category_2          550068 non-null  int64 
 8   Product_Category_3          550068 non-null  int64 
 9   Purchase                    550068 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 42.0+ MB


* Convert to Integer
 1. Age
 2. Stay_In_Current_Years
 

* Hot Encode
 1. Gender
 2. City Category

In [17]:
# convert to int

In [18]:
df_app1['Age'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [19]:
df_app1['Age'] = df_app1['Age'].replace(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'], [0, 1, 2, 3, 4, 5, 6])

In [20]:
df_app1['Age'].unique()

array([0, 6, 2, 4, 5, 3, 1])

In [21]:
df_app1['Stay_In_Current_City_Years'].unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [22]:
df_app1['Stay_In_Current_City_Years'] = df_app1['Stay_In_Current_City_Years'].replace('4+', '4')
df_app1['Stay_In_Current_City_Years'] = df_app1['Stay_In_Current_City_Years'].astype('int64')

In [23]:
df_app1['Stay_In_Current_City_Years'].unique()

array([2, 4, 3, 1, 0])

In [24]:
df_app1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Gender                      550068 non-null  object
 1   Age                         550068 non-null  int64 
 2   Occupation                  550068 non-null  int64 
 3   City_Category               550068 non-null  object
 4   Stay_In_Current_City_Years  550068 non-null  int64 
 5   Marital_Status              550068 non-null  int64 
 6   Product_Category_1          550068 non-null  int64 
 7   Product_Category_2          550068 non-null  int64 
 8   Product_Category_3          550068 non-null  int64 
 9   Purchase                    550068 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 42.0+ MB


In [25]:
df_app1.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,F,0,10,A,2,0,3,0,0,8370
1,F,0,10,A,2,0,1,6,14,15200
2,F,0,10,A,2,0,12,0,0,1422
3,F,0,10,A,2,0,12,14,0,1057
4,M,6,16,C,4,0,8,0,0,7969


In [26]:
from sklearn.model_selection import train_test_split 

X = df_app1.drop(columns=['Purchase'])
y = pd.DataFrame(df_app1['Purchase'], columns=['Purchase'])

In [27]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

X['Gender'] = label_encoder.fit_transform(X['Gender'])
X['City_Category'] = label_encoder.fit_transform(X['City_Category'])

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

reg = Pipeline([
                ('ohe', OneHotEncoder(handle_unknown='ignore')),
                ('model', RandomForestRegressor( n_jobs=-1, verbose = 2)),
])

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [33]:
reg.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 26.9min


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


KeyboardInterrupt: ignored

In [ ]:
reg.score(X_test, y_test)

In [ ]:
y=predreg.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(reg, X, y, cv=5, scoring="neg_mean_squared_error", verbose =1)

In [ ]:
score